In [2]:
from rapidfuzz import fuzz, process, utils
import pandas as pd

In [3]:
dataset = pd.read_csv('../data/CleanData.csv')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7136 entries, 0 to 7135
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Product Name             7136 non-null   object 
 1   Selling Price($)         7136 non-null   float64
 2   About Product            7136 non-null   object 
 3   Product Specification    7136 non-null   object 
 4   Shipping Weight(Pounds)  7136 non-null   float64
 5   Main Category            7136 non-null   object 
 6   Sub-Category             7136 non-null   object 
 7   Side Category            6155 non-null   object 
dtypes: float64(2), object(6)
memory usage: 446.1+ KB


In [4]:
print(dataset['Product Name'][6305])

Mandala


In [5]:
products_list = dataset["Product Name"]
input = "Space Base"
limit = 2
ratio = process.extract(input, products_list, scorer=fuzz.ratio, limit=limit, processor=utils.default_process)
p_ratio = process.extract(input, products_list, scorer=fuzz.partial_ratio, limit=limit, processor=utils.default_process)
tsort_ratio = process.extract(input, products_list, scorer=fuzz.token_sort_ratio, limit=limit, processor=utils.default_process)
ptsort_ratio = process.extract(input, products_list, scorer=fuzz.partial_token_sort_ratio, limit=limit, processor=utils.default_process)
tset_ratio = process.extract(input, products_list, scorer=fuzz.token_set_ratio, limit=limit, processor=utils.default_process)
ptset_ratio = process.extract(input, products_list, scorer=fuzz.partial_token_set_ratio, limit=limit, processor=utils.default_process)
W_ratio = process.extract(input, products_list, scorer=fuzz.WRatio, limit=limit, processor=utils.default_process)
Q_ratio = process.extract(input, products_list, scorer=fuzz.QRatio, limit=limit, processor=utils.default_process)

Scorer_list = {
    'ratio':ratio, 
    'partial ratio':p_ratio, 
    'token sort ratio':tsort_ratio, 
    'partial token sort ratio':ptsort_ratio, 
    'token set ratio':tset_ratio, 
    'partial token set ratio':ptset_ratio, 
    'Wratio':W_ratio, 
    'Qratio':Q_ratio,
}

pd.set_option('display.width', 1000)  
pd.set_option('display.max_colwidth', 60) 
pd.set_option('display.float_format', '{:.2f}'.format)  

df = pd.DataFrame(columns=['Scorer', 'Product Name', 'Value'])
for key,value in Scorer_list.items():
    for i in range(limit):
        new_row = pd.DataFrame([[key, value[i][0], value[i][1]]], columns=['Scorer', 'Product Name', 'Value'])
        df = pd.concat([df, new_row], ignore_index=True)

print(df)

                      Scorer                                                 Product Name  Value
0                      ratio                                                   Space Base 100.00
1                      ratio                                                  Specter Ops  57.14
2              partial ratio                       Space Base: The Emergence of Shy Pluto 100.00
3              partial ratio                                                   Space Base 100.00
4           token sort ratio                                                   Space Base 100.00
5           token sort ratio                                                 Clank!, Base  60.00
6   partial token sort ratio                                                   Space Base 100.00
7   partial token sort ratio                                     Lego Space & Airport Set  84.21
8            token set ratio                       Space Base: The Emergence of Shy Pluto 100.00
9            token set ratio  

C:\Users\Nytoy\AppData\Local\Temp\ipykernel_18676\2016948444.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row], ignore_index=True)


In [6]:
def prototype_rapid_fuzz_filter(user_input, products, number_of_rec):
    list_of_rec = []
    token_set_ratio_match = process.extract(user_input, products, scorer=fuzz.token_set_ratio, limit=1, processor=utils.default_process)
    partial_ratio_matches = process.extract(user_input, products, scorer=fuzz.partial_ratio, limit=number_of_rec, processor=utils.default_process)

    if token_set_ratio_match[0][1] == 100:
        list_of_rec.extend(token_set_ratio_match)
        for match in partial_ratio_matches:
            if match[0] != token_set_ratio_match[0][0]:
                list_of_rec.append(match)
    
    else:
        for match in partial_ratio_matches:
            list_of_rec.append(match)

    return list_of_rec

def rapid_fuzz_rec_to_df(recommendations, dataframe):
    sorted_indeces = [match[2] for match in recommendations]
    reordered_df = dataframe.iloc[sorted_indeces].reset_index(drop=True)
    return reordered_df

In [7]:
print(dataset['Product Name'][3496])

Nanoblock Birthday Cake Building Kit


In [8]:
input = 'Action Toys'
list_recs = []
limit = 5
for i in range(limit):
    print(prototype_rapid_fuzz_filter(user_input=input, products=products_list, number_of_rec=limit)[i])
list_recs = prototype_rapid_fuzz_filter(user_input=input, products=products_list, number_of_rec=len(dataset['Product Name']))


('Transformers Toys Cyberverse Action Attackers: 1-Step Changer Skybyte Action Figure - Repeatable Driller Drive Action Attack - for Kids Ages 6 & Up, 4.25"', 100.0, 138)
('Junior Learning Blend Readers Fiction Toy', 90.0, 1904)
("K'NEX Education ‒ STEM Explorations: Roller Coaster Building Set – 546 Pieces – Ages 8+ Construction Education Toy", 90.0, 2856)
('Galt Toys, First Octons, Construction Toy', 90.0, 5601)
('K’NEX – Turbo Jet – 2-in-1 Building Set – 402 Pieces – Ages 7+ – Engineering Educational Toy', 81.81818181818181, 179)
